In [28]:
import langchain
import os

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [30]:
try:
    with open('../gem_apikey.txt') as f:
        api_key = f.read().strip()
        print("Info: Archivo '../gem_apikey.txt'. OK.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'. Asegúrate de que la ruta es correcta.")
    api_key = None

Info: Archivo '../gem_apikey.txt'. OK.


In [ ]:
def procesar_y_guardar_vectordb():
    """
    Lee un documento PDF, lo procesa para crear una base de datos vectorial
    y la guarda permanentemente en el disco.
    """
    if not api_key:
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- CONFIGURACIÓN ---
    RUTA_PDF = "../database/raw/Doc1.pdf"
    CARPETA_GUARDADO = "../database/indexed/faiss/faiss_index_doc1"
    
    # 1. Carga del documento PDF
    print(f"Cargando el documento PDF desde: {RUTA_PDF}")
    try:
        loader = PyPDFLoader(RUTA_PDF)
        documents = loader.load()
        print(f"Documento cargado. Número de páginas: {len(documents)}")
    except Exception as e:
        print(f"Error al cargar el PDF: {e}")
        return

    # 2. División del texto en fragmentos
    print("Dividiendo el texto en fragmentos...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    print(f"Documento dividido en {len(texts)} fragmentos.")

    # 3. Creación de Embeddings y la base de datos vectorial (en memoria)
    print("Creando embeddings y la base de datos vectorial con Gemini...")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vectorstore = FAISS.from_documents(texts, embeddings)
    print("Base de datos vectorial creada en memoria.")

    # 4. GUARDADO EN DISCO (Este es el paso nuevo y clave)
    print(f"Guardando la base de datos en la carpeta: '{CARPETA_GUARDADO}'...")
    vectorstore.save_local(CARPETA_GUARDADO)
    print("\n¡Éxito! La base de datos vectorial se ha guardado permanentemente.")
    
if __name__ == "__main__":
    procesar_y_guardar_vectordb()

Cargando el documento PDF desde: ../Docs/Doc2.pdf
Documento cargado. Número de páginas: 29
Dividiendo el texto en fragmentos...
Documento dividido en 26 fragmentos.
Creando embeddings y la base de datos vectorial con Gemini...
Base de datos vectorial creada en memoria.
Guardando la base de datos en la carpeta: 'faiss_index_doc2'...

¡Éxito! La base de datos vectorial se ha guardado permanentemente.
